<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/Metadata_Extraction_and_Chunking_05_11_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
%pip install "unstructured[all-docs]" unstructured-client watermark

In [3]:
%load_ext watermark

In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [5]:
from IPython.display import display, JSON

import json

from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError

from unstructured.partition.auto import partition
from unstructured.staging.base import dict_to_elements, elements_to_json

In [6]:
%watermark --iversions

unstructured_client: 0.27.0
json               : 2.0.9
IPython            : 7.34.0
unstructured       : 0.16.4



In [7]:
import unstructured.partition

help(unstructured.partition)

Help on package unstructured.partition in unstructured:

NAME
    unstructured.partition

PACKAGE CONTENTS
    api
    auto
    common (package)
    csv
    doc
    docx
    email
    epub
    html (package)
    image
    json
    md
    model_init
    msg
    odt
    org
    pdf
    pdf_image (package)
    ppt
    pptx
    rst
    rtf
    strategies
    text
    text_type
    tsv
    utils (package)
    xlsx
    xml

FILE
    /usr/local/lib/python3.10/dist-packages/unstructured/partition/__init__.py




In [8]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (270 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [9]:
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 4s (1,212 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123653 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [10]:
!pip install -U nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.4 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [11]:
import nltk
nltk.download('punkt', download_dir='/root/nltk_data')
nltk.download('punkt_tab', download_dir='/root/nltk_data')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [12]:
import nltk
print(nltk.__version__) #must be 3.9.1

3.9.1


In [13]:
%time
from unstructured.partition.auto import partition

# Specify the path to your PDF file
filename = "/content/winter-sports.pdf"

# Call the partition_pdf function
# Returns a List[Element] present in the pages of the parsed pdf document
response = partition(filename)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.34 µs


In [14]:
response

In [15]:
len(response)

901

In [16]:
element_dict = [el.to_dict() for el in response]
print(json.dumps(element_dict, indent=2))

[
  {
    "type": "ListItem",
    "element_id": "acca646a0bcba83f0edb82ce19dc4949",
    "text": "A Distributed Proofreaders US Ebook *",
    "metadata": {
      "coordinates": {
        "points": [
          [
            91.99219214649183,
            152.85352164572794
          ],
          [
            91.99219214649183,
            164.85352191372795
          ],
          [
            372.83790329871175,
            164.85352191372795
          ],
          [
            372.83790329871175,
            152.85352164572794
          ]
        ],
        "system": "PixelSpace",
        "layout_width": 612,
        "layout_height": 792
      },
      "file_directory": "/content",
      "filename": "winter-sports.pdf",
      "languages": [
        "eng"
      ],
      "last_modified": "2024-11-05T17:01:04",
      "page_number": 2,
      "filetype": "application/pdf"
    }
  },
  {
    "type": "NarrativeText",
    "element_id": "6ec7739f853217b1a845380ed110edae",
    "text": "This eb

In [17]:
unique_types = set()

for item in element_dict:
    unique_types.add(item['type'])

print(unique_types)

{'NarrativeText', 'Title', 'UncategorizedText', 'ListItem'}


In [18]:
#Find elements associated with Chapters
[x for x in element_dict if x['type'] == 'Title' and 'winter' in x['text'].lower()]

[{'type': 'Title',
  'element_id': 'bb6a1f21f108211c514bae2404d2f742',
  'text': 'Title: Winter Sports in Switzerland',
  'metadata': {'coordinates': {'points': ((91.99219214649183,
      517.353529786227),
     (91.99219214649183, 529.3535300542271),
     (351.22852985610257, 529.3535300542271),
     (351.22852985610257, 517.353529786227)),
    'system': 'PixelSpace',
    'layout_width': 612,
    'layout_height': 792},
   'file_directory': '/content',
   'filename': 'winter-sports.pdf',
   'languages': ['eng'],
   'last_modified': '2024-11-05T17:01:04',
   'page_number': 2,
   'filetype': 'application/pdf'}},
 {'type': 'Title',
  'element_id': '031880f42be0b69d905158f340cc9bcf',
  'text': 'WINTER SPORTS IN SWITZERLAND',
  'metadata': {'coordinates': {'points': ((179.77735035702676,
      97.98925969642653),
     (179.77735035702676, 163.9892611704264),
     (432.22852749510224, 163.9892611704264),
     (432.22852749510224, 97.98925969642653)),
    'system': 'PixelSpace',
    'layout_w

In [19]:
chapters = [
    "THE SUN-SEEKER",
    "RINKS AND SKATERS",
    "TEES AND CRAMPITS",
    "ICE-HOCKEY",
    "SKI-ING",
    "NOTES ON WINTER RESORTS",
    "FOR PARENTS AND GUARDIANS",
]

In [20]:
chapter_ids = {}
for element in element_dict:
    for chapter in chapters:
        if element["text"] == chapter and element["type"] == "Title":
            chapter_ids[element["element_id"]] = chapter
            break

In [21]:
chapter_ids

{'08ef4b916c44e72dfa21a3b462928b3c': 'THE SUN-SEEKER',
 '83004b57981390b5052bbc077c677e9b': 'RINKS AND SKATERS',
 'd0428d4e418c70676674f93921543ba7': 'TEES AND CRAMPITS',
 'e99335bc0cc4901e83f4ed51b34777a8': 'ICE-HOCKEY',
 '36615698e751eadf28b48d0c96f0384f': 'SKI-ING',
 '06f0ced6df29297c075035fc648700e9': 'NOTES ON WINTER RESORTS',
 '6db9b47269253bef22dfb34790aad59c': 'FOR PARENTS AND GUARDIANS'}

In [22]:
chapter_to_id = {v: k for k, v in chapter_ids.items()}
[x for x in element_dict if x["metadata"].get("parent_id") == chapter_to_id["ICE-HOCKEY"]][0]

{'type': 'NarrativeText',
 'element_id': '295a76c9a7337b0970f95bc27bdee43a',
 'text': 'M\x00\x00\x00 of the Swiss winter-resorts can put into the field a very strong ice-hockey team, and fine teams from other countries often make winter tours there; but the ice-hockey which the ordinary winter visitor will be apt to join in will probably be of the most elementary and unscientific kind indulged in, when the skating day is drawing to a close, by picked-up sides. As will be readily understood, the ice over which a hockey match has been played is perfectly useless for skaters any more that day until it has been swept, scraped, and sprinkled or flooded; and in consequence, at all Swiss resorts, with the exception of St. Moritz, where there is a rink that has been made for the hockey- player, or when an important match is being played, this sport is supplementary to such others as I have spoken of. Nobody, that is, plays hockey and nothing else, since he cannot play hockey at all till the gr

In [23]:
#Load documents into a Vector DB
%pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.0 MB/s eta 0:00

In [24]:
import chromadb

In [25]:
client = chromadb.PersistentClient(path="chroma_tmp", settings=chromadb.Settings(allow_reset=True))
client.reset()

True

In [26]:
collection = client.create_collection(
    name="winter_sports",
    metadata={"hnsw:space": "cosine"}
)

In [27]:
for element in element_dict:
    parent_id = element["metadata"].get("parent_id")
    chapter = chapter_ids.get(parent_id, "")
    collection.add(
        documents=[element["text"]],
        ids=[element["element_id"]],
        metadatas=[{"chapter": chapter}]
    )

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 87.5MiB/s]


In [28]:
#Let's see the elements in VectorDB
results = collection.peek()
print(results["documents"])

['A Distributed Proofreaders US Ebook *', 'This ebook is made available at no cost and with very few restrictions. These restrictions apply only if (1) you make a change in the ebook (other than alteration for different display devices), or (2) you are making commercial use of the ebook. If either of these conditions applies, please check with an FP administrator before proceeding.', "This work is in the Canadian public domain, but may be under copyright in some countries. If you live outside Canada, check your country's copyright laws. If the book is under copyright in your country, do not download or redistribute this file.", 'Title: Winter Sports in Switzerland', 'Author: Benson, E. F. (Edward Frederic)', 'Date of first publication: 1913', 'Date first posted: August 23, 2019', 'Date last updated: February 3, 2021', 'Faded Page ebook#20210233', 'Produced by Chuck Greif and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images available at

In [29]:
# do hybrid search with metadata
result = collection.query(
    query_texts=["How many players are on a team?"],
    n_results=2,
    where={"chapter": "ICE-HOCKEY"},
)
print(json.dumps(result, indent=2))

{
  "ids": [
    [
      "081a282afc9095fbc2c76596594c0490",
      "4e83445779c2acfec3d440845954003a"
    ]
  ],
  "embeddings": null,
  "documents": [
    [
      "accuracy of a first-rate team, each member of which knows the play of the other five players. The finer the team, as is always the case, the greater is their interdependence on each other, and the less there is of individual play. Brilliant running and dribbling, indeed, you will see; but as distinguished from a side composed of individuals, however good, who are yet not a team, these brilliant episodes are always part of a plan, and end not in some wild shot but in a pass or a succession of passes, designed to lead to a good opening for scoring. There is, indeed, no game at which team play outwits individual brilliance so completely.",
      "quickly, and above all, stop quickly. However swift a player may be, he is practically useless to his side unless he can, with moderate suddenness, check his headlong career, turn qui

In [30]:
result = collection.query(
    query_texts=["What is Jangfrau?"],
    n_results=2,
    where={"chapter": "ICE-HOCKEY"},
)
print(json.dumps(result, indent=2))

{
  "ids": [
    [
      "999af189fe150b8c751e9dd189c6d901",
      "b70995ddbd9b4f00b95853ac953cbe29"
    ]
  ],
  "embeddings": null,
  "documents": [
    [
      "It is a wonderful and delightful sight to watch the speed and",
      "But such is not the aspect of the game that will strike the observer who watches the usual pick-up or inter-hotel match on the rink, which generally begins as soon as skaters hear the curfew of the tea-bell. Here will be found the individualist who, sooner than pass when he has once got the puck, would infinitely prefer to fall and be trampled on; and you will see him, while still sitting on the ice, hacking wildly at the beloved india-rubber, in flat contravention of the rule. Common, too, are the \u201cnon-stops\u201d (like Wimbledon trains) who, once having got up speed, are practically brakeless. Indeed, it was in connection with non-stops that the present writer saw the most ludicrously comic incident that it has ever been his good luck to encounter

In [31]:
result = collection.query(
    query_texts=["What is Jangfrau?"],
    n_results=2,
    where={"chapter": "THE SUN-SEEKER"},
)
print(json.dumps(result, indent=2))

{
  "ids": [
    [
      "a2313f32eead49a43c4d1e41066bdeac",
      "6c645fef54fc43754627dbe221205b96"
    ]
  ],
  "embeddings": null,
  "documents": [
    [
      "The Jungfrau is and will always be my mistress among mountains,",
      "as a first charge, and after they have been stowed, the mere necessities of life, like clothes and dressing-case, as opposed to its joys, fill the rest. Even in the harbour the steamer sways with the back-wash of the heavy seas outside, and the mooring-ropes squeak and strain to its unease. I stick in the narrow gang-plank that conducts in precipitous incline to the deck (at least the corner of my suit-case does, which is part of my identity); a faint and awful smell of red plush sofas and cold beef comes up from the stairs leading to the saloon; the tarpaulins, rigged up along the open passage between decks, flap uneasily and are buffeted by the rain-soaked wind, and sailors hurry about with white japanned tin objects in their hands...."
    ]
  ],
  

In [32]:
# without providing the specific chapter to go for
result = collection.query(
    query_texts=["What is Jangfrau?"],
    n_results=2,
    #where={"chapter": "THE SUN-SEEKER"},
)
print(json.dumps(result, indent=2))

{
  "ids": [
    [
      "a2313f32eead49a43c4d1e41066bdeac",
      "f22253bd2860aca62523bd31739c3207"
    ]
  ],
  "embeddings": null,
  "documents": [
    [
      "The Jungfrau is and will always be my mistress among mountains,",
      "CHAPTER VII"
    ]
  ],
  "uris": null,
  "data": null,
  "metadatas": [
    [
      {
        "chapter": "THE SUN-SEEKER"
      },
      {
        "chapter": ""
      }
    ]
  ],
  "distances": [
    [
      0.599014401435852,
      0.7450516819953918
    ]
  ],
  "included": [
    "distances",
    "documents",
    "metadatas"
  ]
}


In [33]:
#do Chunking by Title
#Instead of the de-facto chunking we can dictate how to do the chunking of documents.
from unstructured.chunking.title import chunk_by_title

In [34]:
chunks = chunk_by_title(
    response,
    combine_text_under_n_chars=100,
    max_characters=3000,
)

In [35]:
JSON(json.dumps(chunks[0].to_dict(), indent=2))

<IPython.core.display.JSON object>

In [36]:
len(element_dict)

901

In [37]:
len(chunks)

199